In [23]:
import pandas as pd

In [24]:
#import annotazioni e gold standard
annotazioni= pd.read_csv('files/annotazioni_full.csv', delimiter=',')
GS= pd.read_csv('files/gold_standard.csv', delimiter=',')

# descrizione problema

In [26]:
annotazioni.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'annotator'],
      dtype='object')

In [27]:
GS.columns

Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui'],
      dtype='object')

In [28]:
#allineazione nomi
annotazioni = annotazioni.rename(columns={'ontologies':'Ontologies','SOURCE':'Source','entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})
annotazioni['Ontologies'] = annotazioni['Ontologies'].replace('SNOMEDCT', 'SNOMED-CT')

GS = GS.rename(columns={'entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})

In [29]:

set_gold = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_full = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Calcoliamo i valori richiesti
Nboth = len(set_gold & set_full)  # Presenti in entrambi
Nleft = len(set_gold - set_full)  # Solo in df_gold
Nright = len(set_full - set_gold)  # Solo in df_full

print(f"Nboth: {Nboth}, Nleft: {Nleft}, Nright: {Nright}")

Nboth: 58, Nleft: 34, Nright: 1345


In [31]:
#dettagliare questa analisi rispetto alle diverse ontologie

set_GS = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_annotazioni = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Unione dei due DataFrame per analisi per Ontology
df_combined = pd.concat([GS.assign(source='GS'), annotazioni.assign(source='annotazioni')])

# Creiamo una tabella che conta Nleft, Nright e Nboth per ogni Ontology
ontology_counts = df_combined.groupby(['Ontologies', 'source']).size().unstack(fill_value=0)
ontology_counts.columns = ['Nleft', 'Nright']
ontology_counts['Nboth'] = ontology_counts.min(axis=1)  # Il minimo tra Nleft e Nright dà Nboth

# Output finale
print(ontology_counts)

            Nleft  Nright  Nboth
Ontologies                      
LOINC           0    7903      0
SNOMED-CT     127   20398    127
UMLS          196    9801    196


In [32]:
#dettagliare questa analisi rispetto alle diverse annotazioni
# Aggiungiamo un annotatore fittizio per GS
GS['Annotator'] = 'Gold_Standard'

# Cambiamo il nome della colonna 'source' in 'Provenienza'
GS['Provenienza'] = 'GS'
annotazioni['Provenienza'] = 'annotazioni'
df_combined = pd.concat([GS, annotazioni])

# Raggruppiamo per Ontology e Annotator
ontology_annotator_counts = df_combined.groupby(['Ontologies', 'Annotator', 'Provenienza']).size().unstack(fill_value=0)

# Rinominiamo le colonne
ontology_annotator_counts.columns = ['Nleft', 'Nright']
ontology_annotator_counts['Nboth'] = ontology_annotator_counts.min(axis=1)

# Output finale
print(ontology_annotator_counts)

                          Nleft  Nright  Nboth
Ontologies Annotator                          
LOINC      Bioportal          0    7903      0
SNOMED-CT  Bioportal          0    4791      0
           Gold_Standard    127       0      0
           MedCAT             0   15607      0
UMLS       Gold_Standard    196       0      0
           MedCAT             0    9801      0


# Valutazione della Qualità degli Annotatori